**Data:** 

- $I$: the set of all nurses. 
- $J$: the set of shifts.
- $k$: the shift inequality.
- $N$: the subset of shifts $J$ that are night shifts
- $M$: the subset of shifts $J$ that are morning shifts
- $E$: the subset of shifts $J$ that are evening shifts
- $P_{ij}$: a categorical variable denoting the preference of nurse $i$ for shift $j$.
- $C_{j}$: number of nurses required to work during shift $j$

**Decision Variables:** 
For each nurse $i \in I$ and for each shift $j \in J$,let $x_{ij}$ denote whether the nurse is scheduled or not.



**Auxilary Decision Variables:** 
- $h$: the highest number of shifts for a nurse
- $l$: the lowest number of shifts for a nurse
- $ma$: the highest number of shifts for a nurse at night
- $mi$: the lowest number of shifts for a nurse at night

**Objective and constraints:**

$$\begin{aligned}
\text{Maximize:} && \sum_{i \in I, j \in J} P_{ij}*x_{ij} - 100 * (h-l) - 150 *& (ma-mi) \\
\text{subject to:} \\
\text{(Max Number of shifts)} && \sum_{j \in J} x_{ij} & \le 6 &\text{ for each nurse $i \in I$ in each week} &&\\
\text{(Non-consecutive requirement)} && x_{ij} + x_{i(j+1)} &\le 1 &\text{ for each nurse $i \in I$, for each $j \in J$} &&\\
\text{(Shift Requirement)} && \sum_{i \in I,j \in J}x_{ij} &\ge \sum_{j \in J}C_j &&\\
\text{(Night rest)} && (x_{ij-2} + x_{i(j-1)} + x_{ij+1} + x_{i(j+2)}) * x_{ij} &\le 0 &\text{ for each nurse $i \in I$, for each $j$ = night shift}&&\\
\text{(blackout Constraint)} && P_{ij} &\ge x_{ij} &\text{ for each nurse $i \in I$, for each $j \in J$}&&\
\end{aligned}$$

In [1]:
from gurobipy import GRB, Model, max_, min_
import pandas as pd

## Small data

In [43]:
mod = Model()

## reading data
pref = pd.read_excel('small_data.xlsx', header = [0,1,2],sheet_name = 'Preferences')
shift_id = pref.columns.get_level_values(2)
pref_column = pref.columns
pref.columns = shift_id

req = pd.read_excel('small_data.xlsx', sheet_name = 'Requirements', index_col = 0)
req = req.drop(columns = ['day', 'time'],axis =1)


## setting variables
I = pref.index
J = shift_id
x = mod.addVars(I,J, vtype = GRB.BINARY)
h = mod.addVar()
l = mod.addVar()
ma = mod.addVar()
mi = mod.addVar()
shift_num = mod.addVars(I)
night_num = mod.addVars(I)

## setting objective
pref_score = sum(pref.loc[i,j]*x[i,j] for i in I for j in J)
mod.setObjective(pref_score - 100*(h-l) - 150*(ma - mi), sense = GRB.MAXIMIZE)
#mod.setObjective(sum(pref.loc[i,j]*x[i,j] for i in I for j in J), sense = GRB.MAXIMIZE)


## setting Constrs

## define h, l, ma, mi
for i in I:
    mod.addConstr(shift_num[i] == sum(x[i,j] for j in J))

mod.addConstr(h == max_([shift_num[i] for i in I]))
mod.addConstr(l == min_([shift_num[i] for i in I]))

for i in I:
    mod.addConstr(night_num[i] == sum(x[i,((3*j)+2)] for j in range(int(len(J)/3))))

mod.addConstr(ma == max_([night_num[i] for i in I]))
mod.addConstr(mi == min_([night_num[i] for i in I]))

## Constr: no more than six shifts per week
for i in I:
    for week in range(0,int(len(J)),21):
        mod.addConstr(sum(x[i,d] for d in range(week, week+21)) <= 6)

## Constr: no consecutive shift
for i in I:
    for j in range(len(J)-1):
        mod.addConstr(x[i,j] + x[i,(j+1)] <= 1)

## Constr: match the shift requirement       
for j in J:
    mod.addConstr(sum(x[i,j] for i in I) == req.loc[j,'persons'])

## Constr: night shift breaks 
for i in I:
    for n in range(2,(len(J)-3),3):
        mod.addConstr((x[i,n-2]+x[i,n-1]+x[i,n+1]+x[i,n+2])*x[i,n] == 0)
    mod.addConstr((x[i,len(J)-3]+x[i,len(J)-2])*x[i,len(J)-1] == 0)

## Constr: blacked-out
for i in I:
    for j in J:
        mod.addConstr(pref.loc[i,j] - x[i,j] >= 0)

#mod.computeIIS()

mod.setParam('outputflag', False)
mod.optimize()
mod.objval


-667.0

In [44]:
names = pref.index
schedule=pd.DataFrame('',index=names,columns=shift_id)

for i in I:
    for j in J:
        if x[i,j].x == 1:
            schedule.loc[i,j] = x[i,j].x

schedule.columns = pref_column
        
schedule

day         2019-03-31               2019-04-01               2019-04-02  \
time           Morning Evening Night    Morning Evening Night    Morning   
shift_id            0       1     2          3       4     5          6    
name                                                                       
Alexis                                                1                1   
Alyssa               1                                1                    
Anthony                            1                                   1   
Brandon              1                        1                            
Brianna                      1                1                            
Caleb                        1                        1                    
Cameron                      1                                             
Chloe                1                                      1              
Christopher                                   1                        1   

day                       2019-04-03  ...        2019-04-04                \
time        Evening Night    Morning  ...  Night    Morning Evening Night   
shift_id         7     8          9   ...     11         12      13    14   
name                                  ...                                   
Alexis                             1  ...                         1         
Alyssa            1                   ...                 1                 
Anthony                            1  ...                 1                 
Brandon           1                   ...                 1                 
Brianna                            1  ...                                   
Caleb             1                   ...                         1         
Cameron                               ...                               1   
Chloe                   1             ...      1                            
Christopher                           ...                         1         

day         2019-04-05               2019-04-06                
time           Morning Evening Night    Morning Evening Night  
shift_id            15      16    17         18      19    20  
name                                                           
Alexis               1                                1        
Alyssa               1                                         
Anthony                      1                1                
Brandon                      1                        1        
Brianna                                                        
Caleb                                                 1        
Cameron                            1                        1  
Chloe                        1                1                
Christopher          1                        1                

[9 rows x 21 columns]

In [4]:
summary = pd.Series(name='Value')
summary['Objective'] = mod.objval
summary['Total Preference Score'] = sum(x[i,j].x * pref.loc[i,j] for i in I for j in J)
summary['Shift inequality'] = (h.x-l.x)
summary['Night inequality'] = (ma.x-mi.x)
summary

Objective                -667.0
Total Preference Score     83.0
Shift inequality            3.0
Night inequality            3.0
Name: Value, dtype: float64

In [5]:
writer = pd.ExcelWriter('tmp_smalldata.xlsx',datetime_format='m/dd')
schedule.to_excel(writer,sheet_name='Schedule')
summary.to_excel(writer,sheet_name='Summary')
writer.save()

## Real challenge

In [2]:
from gurobipy import GRB, Model, max_, min_
import pandas as pd
mod = Model()

## reading data
pref = pd.read_excel('data.xlsx', header = [0,1,2],sheet_name = 'Preferences')
shift_id = pref.columns.get_level_values(2)
pref_column = pref.columns
pref.columns = shift_id

req = pd.read_excel('data.xlsx', sheet_name = 'Requirements', index_col = 0)
req = req.drop(columns = ['day', 'time'],axis =1)


## setting variables
I = pref.index
J = shift_id
x = mod.addVars(I,J, vtype = GRB.BINARY)
h = mod.addVar()
l = mod.addVar()
ma = mod.addVar()
mi = mod.addVar()
shift_num = mod.addVars(I)
night_num = mod.addVars(I)

## setting objective
pref_score = sum(pref.loc[i,j]*x[i,j] for i in I for j in J)
mod.setObjective(pref_score - 100*(h-l) - 150*(ma - mi), sense = GRB.MAXIMIZE)
#mod.setObjective(sum(pref.loc[i,j]*x[i,j] for i in I for j in J), sense = GRB.MAXIMIZE)


## setting Constrs

## define h, l, ma, mi
for i in I:
    mod.addConstr(shift_num[i] == sum(x[i,j] for j in J))

mod.addConstr(h == max_([shift_num[i] for i in I]))
mod.addConstr(l == min_([shift_num[i] for i in I]))

for i in I:
    mod.addConstr(night_num[i] == sum(x[i,((3*j)+2)] for j in range(int(len(J)/3))))

mod.addConstr(ma == max_([night_num[i] for i in I]))
mod.addConstr(mi == min_([night_num[i] for i in I]))

## Constr: no more than six shifts per week
for i in I:
    for week in range(0,int(len(J)),21):
        mod.addConstr(sum(x[i,d] for d in range(week, week+21)) == 6)

## Constr: no consecutive shift
for i in I:
    for j in range(len(J)-1):
        mod.addConstr(x[i,j] + x[i,(j+1)] <= 1)

## Constr: match the shift requirement       
for j in J:
    mod.addConstr(sum(x[i,j] for i in I) >= req.loc[j,'persons'])

## Constr: night shift breaks 
for i in I:
    for n in range(2,(len(J)-3),3):
        mod.addConstr((x[i,n-2]+x[i,n-1]+x[i,n+1]+x[i,n+2])*x[i,n] == 0)
    mod.addConstr((x[i,len(J)-3]+x[i,len(J)-2])*x[i,len(J)-1] == 0)

## Constr: blacked-out
for i in I:
    for j in J:
        mod.addConstr(pref.loc[i,j] - x[i,j] >= 0)

#mod.computeIIS()

mod.setParam('outputflag', False)
mod.optimize()
mod.objval

Academic license - for non-commercial use only


AttributeError: b"Unable to retrieve attribute 'objval'"

In [3]:
from gurobipy import Model, GRB, max_, min_
import pandas as pd
def optimize(inputFile,outputFile):  
    #read files
    prefs=pd.read_excel(inputFile,header=[0,1,2],sheet_name='Preferences',index_col=0)
    req=pd.read_excel(inputFile,sheet_name='Requirements',index_col=0)
    names=prefs.index
    shifts=prefs.columns
    shift_id=shifts.get_level_values(2)
    prefs.columns=shift_id
    prefs.head()
    
    mod=Model()

    N = len(shift_id) # the total number of shift
    nights = range(2,N,3) # the index of night shifts in shift_id

    # decision vars:
    x = mod.addVars(names,shift_id,vtype=GRB.BINARY) # allocated status for each nurse in each shift
    y = mod.addVars(names) # total shift number for each nurse
    z = mod.addVars(names) # total night shift for each nurse
    shift_max = mod.addVar()
    shift_min = mod.addVar()
    night_max = mod.addVar()
    night_min = mod.addVar()


    # constraints:
    # calculate total shift number of each nurse
    for name in names:
        mod.addConstr(y[name] == sum([x[name,shift] for shift in shift_id]))
    # calculate total night shift of each nurse
    for name in names:
        mod.addConstr(z[name] == sum([x[name,night] for night in nights]))

    # calculate the max, min of shift, night shift
    mod.addConstr(shift_max == max_([y[name] for name in names]))
    mod.addConstr(shift_min == min_([y[name] for name in names]))
    mod.addConstr(night_max == max_([z[name] for name in names]))
    mod.addConstr(night_min == min_([z[name] for name in names]))


    # no nurse 6 shifts in a week
    weeks=range(0,N,21)
    for name in names:
        for week in weeks:
            mod.addConstr(sum([x[name,day] for day in range(week,week+21)])<=6)

    # no consecutive shift
    for name in names:
        for shift in range(N-1):
            mod.addConstr((x[name,shift]+x[name,shift+1])<=1)

    # night shift constraints
    for name in names:
        for night in range(2,N-3,3):
            mod.addConstr((x[name,night-2]+x[name,night-1]+x[name,night+1]+x[name,night+2])*x[name,night]==0)
        mod.addConstr((x[name,N-3]+x[name,N-2])*x[name,N-1]==0)

    # shift requirement
    for shift in shift_id:
        mod.addConstr(sum([x[name,shift] for name in names])==req.loc[shift,'persons'])

    # black out nurse preference
    for name in names:
        for shift in shift_id:
            mod.addConstr(prefs.loc[name,shift]-x[name,shift]>=0)


    #objective
    # sum_of_prefernce_scores
    sum_of_prefernce_scores = sum([prefs.loc[name,shift]*x[name,shift] for name in names for shift in shift_id])

    mod.setObjective(sum_of_prefernce_scores-100*(shift_max-shift_min)-150*(night_max-night_min),sense=GRB.MAXIMIZE)


    mod.optimize()


    #ouputfile
    schedule=pd.DataFrame('',index=names,columns=shift_id)
    for name in names:
        for shift in shift_id:
            if x[name,shift].x==1:
                schedule.loc[name,shift]=x[name,shift].x
    schedule.columns=shifts
    schedule.head()

    summary=pd.Series(name='Value')
    summary['Objective']=mod.objval
    summary['Total preference score']=sum_of_prefernce_scores.getValue()
    summary['Shift inequality']=shift_max.x-shift_min.x
    summary['Night inequality']=night_max.x-night_min.x

    writer=pd.ExcelWriter(outputFile)
    pd.DataFrame(schedule).to_excel(writer,sheet_name='Schedule')
    pd.DataFrame(summary).to_excel(writer,sheet_name='Summary')
    writer.save()

In [4]:
optimize('small_data.xlsx','test.xlsx')

Optimize a model with 417 rows, 211 columns and 1197 nonzeros
Model has 63 quadratic constraints
Model has 4 general constraints
Variable types: 22 continuous, 189 integer (189 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 219 rows and 47 columns
Presolve time: 0.00s
Presolved: 717 rows, 321 columns, 1932 nonzeros
Variable types: 0 continuous, 321 integer (301 binary)

Root relaxation: objective 6.670000e+02, 266 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -667.0000000 -667.00000  0.00%     -    0s

Explored 1 nodes (266 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: -667 

Optimal solution found (to

In [5]:
names = pref.index
schedule2 = pd.DataFrame('',index=names,columns=shift_id)

for i in I:
    for j in J:
        if x[i,j].x == 1:
            schedule2.loc[i,j] = x[i,j].x
            
schedule2.columns = pref_column
            
schedule2

day         2019-03-31               2019-04-01               2019-04-02  \
time           Morning Evening Night    Morning Evening Night    Morning   
shift_id           0       1     2          3       4     5          6     
name                                                                       
Alexis                       1                        1                    
Alyssa                       1                        1                1   
Anthony                            1                        1              
Brandon                                       1                            
Brianna                      1                              1              
Caleb                        1                        1                    
Cameron              1                                                     
Chloe                1                                                 1   
Christopher                                           1                1   
Daniel                             1                                       
David                1                                1                    
Destiny                      1                1                            
Dorothy                                                                    
Ethan                                         1                            
Gabriel                            1                        1              
Hailey                             1                                   1   
Hannah               1                        1                            
Imani                1                        1                            
Isaiah               1                                      1              
Isaac                        1                        1                    
James                                                                  1   
Jasmine              1                                1                1   
Jayden                       1                        1                    
Jayla                1                                1                    
Jeremiah                     1                              1              
Jordan               1                        1                            
Joseph                       1                                         1   
Joshua               1                                                     
Josiah                       1                1                        1   
Justin               1                        1                            
Kayla                        1                1                        1   
Kennedy              1                                1                1   
Kevin                                                 1                1   
Kiara                              1                                       
Madison              1                        1                            
Makayla                      1                                             
Malik                        1                        1                1   
Matthew                            1                                       
Michael                                       1                            
Nathan                                                1                1   
Nevaeh                                        1                        1   
Patricia                     1                              1              
Susan                              1                        1              
Sydney                                        1                            
Taylor                       1                              1              
Tiana                1                        1                            
Trinity                      1                        1                    
Tyler                1                                1                1   
William                            1            

In [6]:
summary2 = pd.Series(name='Value')
summary2['Objective'] = mod.objval
summary2['Total Preference Score'] = sum(x[i,j].x * pref.loc[i,j] for i in I for j in J)
summary2['Shift inequality'] = (h.x-l.x)
summary2['Night inequality'] = (ma.x-mi.x)
summary2

Objective                 3751.0
Total Preference Score    4251.0
Shift inequality             5.0
Night inequality             0.0
Name: Value, dtype: float64

In [7]:
writer2 = pd.ExcelWriter('tmp.xlsx',datetime_format='m/dd')
schedule2.to_excel(writer2,sheet_name='Schedule')
summary2.to_excel(writer2,sheet_name='Summary')
writer2.save()